In [1]:
import numpy as np
import re
import time
from stanfordcorenlp import StanfordCoreNLP
import torch
from torch.autograd import Function
from torch.autograd import Variable
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tqdm import tqdm
import time
import math
import logging
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from sklearn import preprocessing
from random import shuffle
BATCH_SIZE = 128
tl.set_backend('pytorch')
help(tl.set_backend)

Using numpy backend.


Help on function set_backend in module tensorly:

set_backend(backend_name)
    Sets the backend for TensorLy
    
        The backend will be set as specified and operations will used that backend
    
    Parameters
    ----------
    backend_name : {'mxnet', 'numpy', 'pytorch', 'tensorflow', 'cupy'}, default is 'numpy'



Using pytorch backend.


In [2]:
b = np.fromfile('tensors_2010.dat',dtype=np.float64).reshape((8000,-1,200))
Y = np.fromfile('tensors_2010_labels.dat',dtype=np.int)
position_tag_1 = np.fromfile('tensors_2010_entity1.dat',dtype=np.int)
position_tag_2 = np.fromfile('tensors_2010_entity2.dat',dtype=np.int)
position_tag = np.concatenate((position_tag_1,position_tag_2)).reshape((8000,-1))
b.shape,Y.shape,position_tag_1.shape,position_tag_2.shape,position_tag.shape

((8000, 85, 200), (8000,), (8000,), (8000,), (8000, 2))

In [3]:
inputs = []
for i in range(8000):
    inputs.append((torch.from_numpy(b[i]),Y[i]))
shuffle(inputs)
flag = int(len(inputs)*0.7)
train = inputs[:flag]
test = inputs[flag:]

In [4]:
t_e = []
t_y = []
for E,Y in test:
    t_e.append(E)
    t_y.append(Y)
t_e = torch.stack(t_e)
t_y = torch.from_numpy(np.array(t_y))
t_e.shape, t_y.shape

(torch.Size([2400, 85, 200]), torch.Size([2400]))

# glove_rnn_max: 0.6158333333333333 4轮

In [5]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class RNN_max(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.GRU(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )
        self.out=torch.nn.Linear(in_features=64,out_features=10)
        self.maxpool = torch.nn.MaxPool2d((85,1))

    def forward(self,x):
        output,h_n=self.rnn(x.float())
        output_in_last_timestep = self.maxpool(output)
        output_in_last_timestep = output_in_last_timestep.reshape((output_in_last_timestep.shape[0],-1))
        x=self.out(output_in_last_timestep)
        return x

In [ ]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = RNN_max()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,301): 
    for step, (batch_x, batch_y) in enumerate(loader):
        pred = net(batch_x)
        loss = loss_F(pred,batch_y) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

RNN_max(
  (rnn): GRU(200, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
  (maxpool): MaxPool2d(kernel_size=(85, 1), stride=(85, 1), padding=0, dilation=1, ceil_mode=False)
)
1: accuracy:0.17375 loss: 2.3217885494232178
2: accuracy:0.5570833333333334 loss: 1.1998502016067505
3: accuracy:0.58 loss: 0.9416068196296692
4: accuracy:0.6158333333333333 loss: 0.7349043488502502
5: accuracy:0.6058333333333333 loss: 0.5695826411247253
6: accuracy:0.6054166666666667 loss: 0.38526633381843567
7: accuracy:0.5920833333333333 loss: 0.24833987653255463
8: accuracy:0.6029166666666667 loss: 0.2028799206018448
9: accuracy:0.5916666666666667 loss: 0.11811262369155884
10: accuracy:0.6 loss: 0.06206924840807915
11: accuracy:0.6033333333333334 loss: 0.043837450444698334
12: accuracy:0.5833333333333334 loss: 0.027932940050959587
13: accuracy:0.5983333333333334 loss: 0.015282370150089264
14: accuracy:0.59875 loss: 0.009410955011844635
15: accuracy:0.5991666666666666 loss: 0

144: accuracy:0.5641666666666667 loss: 0.002643834799528122
145: accuracy:0.5654166666666667 loss: 0.0026444606482982635
146: accuracy:0.5633333333333334 loss: 0.0025319792330265045
147: accuracy:0.5641666666666667 loss: 0.002333667129278183
148: accuracy:0.5641666666666667 loss: 0.0017496980726718903
149: accuracy:0.56375 loss: 0.0022230856120586395
150: accuracy:0.5654166666666667 loss: 0.001850184053182602
151: accuracy:0.5645833333333333 loss: 0.0017025582492351532
152: accuracy:0.5666666666666667 loss: 0.0016517601907253265
153: accuracy:0.5608333333333333 loss: 0.0017395354807376862
154: accuracy:0.5633333333333334 loss: 0.0013730302453041077
155: accuracy:0.5658333333333333 loss: 0.0019898563623428345
156: accuracy:0.5616666666666666 loss: 0.0015384480357170105
157: accuracy:0.5591666666666667 loss: 0.0013953819870948792
158: accuracy:0.56625 loss: 0.0016397014260292053
159: accuracy:0.5675 loss: 0.0011992249637842178
160: accuracy:0.5616666666666666 loss: 0.0015268884599208832


# glove_bi-rnn_max: 0.6158333333333333 4轮

In [7]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class bi_RNN_max(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.GRU(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
            bidirectional=True
        )
        self.out=torch.nn.Linear(in_features=128,out_features=10)
        self.maxpool = torch.nn.MaxPool2d((85,1))

    def forward(self,x):
        output,h_n=self.rnn(x.float())
        output_in_last_timestep = self.maxpool(output)
        output_in_last_timestep = output_in_last_timestep.reshape((output_in_last_timestep.shape[0],-1))
        x=self.out(output_in_last_timestep)
        return x

In [ ]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = bi_RNN_max()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,301): 
    for step, (batch_x, batch_y) in enumerate(loader):
        pred = net(batch_x)
        loss = loss_F(pred,batch_y) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

bi_RNN_max(
  (rnn): GRU(200, 64, batch_first=True, bidirectional=True)
  (out): Linear(in_features=128, out_features=10, bias=True)
  (maxpool): MaxPool2d(kernel_size=(85, 1), stride=(85, 1), padding=0, dilation=1, ceil_mode=False)
)
1: accuracy:0.18333333333333332 loss: 2.219050407409668
2: accuracy:0.5733333333333334 loss: 0.9770408272743225
3: accuracy:0.6091666666666666 loss: 0.8143296837806702
4: accuracy:0.6216666666666667 loss: 0.5164687633514404
5: accuracy:0.6066666666666667 loss: 0.3578956723213196
6: accuracy:0.6266666666666667 loss: 0.164963036775589
7: accuracy:0.6045833333333334 loss: 0.09574701637029648
8: accuracy:0.61875 loss: 0.03480490669608116
9: accuracy:0.6195833333333334 loss: 0.02018904685974121
10: accuracy:0.61625 loss: 0.009954210370779037
11: accuracy:0.6216666666666667 loss: 0.007833383977413177
12: accuracy:0.6183333333333333 loss: 0.006788436323404312
13: accuracy:0.6183333333333333 loss: 0.005218572914600372
14: accuracy:0.6191666666666666 loss: 0.01804

# glove:0.44680851063829785

In [9]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class RNN_old(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.GRU(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )        
        self.out=torch.nn.Linear(in_features=64,out_features=10)

    def forward(self,x):
        output,h_n=self.rnn(x.float())
        output_in_last_timestep=h_n[-1,:,:]
        x=self.out(output_in_last_timestep)
        return x

In [10]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = RNN_old()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,251): 
    for step, (batch_x, batch_y) in enumerate(loader):
        pred = net(batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

RNN_old(
  (rnn): GRU(200, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)
1: accuracy:0.12056737588652482 loss: 2.2877795696258545
2: accuracy:0.12056737588652482 loss: 2.292205333709717
3: accuracy:0.12056737588652482 loss: 2.203953742980957
4: accuracy:0.12056737588652482 loss: 2.2441458702087402
5: accuracy:0.12056737588652482 loss: 2.2280540466308594
6: accuracy:0.12056737588652482 loss: 2.223677396774292
7: accuracy:0.12056737588652482 loss: 2.2504940032958984
8: accuracy:0.12056737588652482 loss: 2.2234230041503906
9: accuracy:0.12056737588652482 loss: 2.2240872383117676
10: accuracy:0.12056737588652482 loss: 2.212447166442871
11: accuracy:0.12056737588652482 loss: 2.248894691467285
12: accuracy:0.12056737588652482 loss: 2.1920266151428223
13: accuracy:0.12056737588652482 loss: 2.232753038406372
14: accuracy:0.12056737588652482 loss: 2.218202590942383
15: accuracy:0.12056737588652482 loss: 2.3168013095855713
16: accuracy:0.12056737588652482 lo

142: accuracy:0.23404255319148937 loss: 0.22720927000045776
143: accuracy:0.2765957446808511 loss: 0.12546999752521515
144: accuracy:0.24113475177304963 loss: 0.457686185836792
145: accuracy:0.2907801418439716 loss: 0.1770227551460266
146: accuracy:0.2765957446808511 loss: 0.05575019121170044
147: accuracy:0.28368794326241137 loss: 0.10316809266805649
148: accuracy:0.2695035460992908 loss: 0.1405351310968399
149: accuracy:0.2624113475177305 loss: 0.09492642432451248
150: accuracy:0.2907801418439716 loss: 0.057461269199848175
151: accuracy:0.2978723404255319 loss: 0.05150166153907776
152: accuracy:0.3120567375886525 loss: 0.09667906910181046
153: accuracy:0.2907801418439716 loss: 0.03867962583899498
154: accuracy:0.2978723404255319 loss: 0.10656842589378357
155: accuracy:0.2553191489361702 loss: 0.565399169921875
156: accuracy:0.24113475177304963 loss: 0.6753730773925781
157: accuracy:0.2553191489361702 loss: 0.6518615484237671
158: accuracy:0.2198581560283688 loss: 0.5570526123046875
1

# N-new 0.6737588652482269 17轮

In [5]:
inputs = []
MAX_LENGTH = 53
for i in range(0,467):
    a = np.loadtxt('/Users/denhiroshi/Downloads/Embeding/ans_1.0_'+str(i)+'.csv',dtype=np.float64,delimiter=',')
    a = a.transpose(1,0)
    a = np.exp(a)
    a = np.sum(a,axis=1)
    a = np.log(a)
    if a.shape[0] < MAX_LENGTH:
        add = MAX_LENGTH - a.shape[0]
        adds = np.zeros(add)
        a = np.concatenate((a, adds))
    inputs.append(a)

In [6]:
use = np.stack(inputs)
mean = use.mean()         #计算平均数
deviation = use.std()     #计算标准差
# 标准化数据的公式: (数据值 - 平均数) / 标准差
use = (use - mean) / deviation

In [7]:
b = np.fromfile('glove_WV.dat',dtype=np.float64).reshape((467,-1,200))

In [8]:
b.shape,use.shape

((467, 53, 200), (467, 53))

In [9]:
inputs = []
for i in range(467):
    inputs.append((torch.from_numpy(b[i]),torch.from_numpy(use[i]),Y[i]))
shuffle(inputs)
flag = int(len(inputs)*0.7)
train = inputs[:flag]
test = inputs[flag:]

In [10]:
t_s = []
t_e = []
t_y = []
for E,S,Y in test:
    t_s.append(S)
    t_e.append(E)
    t_y.append(Y)
t_s = torch.stack(t_s)
t_e = torch.stack(t_e)
t_y = torch.from_numpy(np.array(t_y))
t_s.shape, t_e.shape, t_y.shape

(torch.Size([141, 53]), torch.Size([141, 53, 200]), torch.Size([141]))

In [11]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class RNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.GRU(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )
        self.out=torch.nn.Linear(in_features=65 ,out_features=7)
        self.maxpool = torch.nn.MaxPool2d((53,1))

    def forward(self,sen,x):
        output,h_n=self.rnn(x.float())
        sen = sen.reshape((sen.shape[0],sen.shape[1],1))  # (BATCH_SIZE,53,1)
        output1 = torch.bmm(torch.transpose(output,1,2).float(),sen.float())  # (BATCH_SIZE,64,1)
        output1 = torch.sigmoid(output1)
        output1 = torch.softmax(output1,1)
        output1 = torch.bmm(output,output1) # (BATCH_SIZE,53,1)
        output = torch.cat((output1,output),2) # (BATCH_SIZE,53,65)
        output = self.maxpool(output)   # (BATCH_SIZE,1,65)
        output = output.reshape((output.shape[0],-1)) # (BATCH_SIZE,7)
        x1 = self.out(output.float())
        return x1

In [15]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
    drop_last=True,
)
net = RNN()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,201): 
    for step, (batch_x, sen, batch_y) in enumerate(loader):
        pred = net(sen, batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 1: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_s, t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

RNN(
  (rnn): GRU(200, 64, batch_first=True)
  (out): Linear(in_features=65, out_features=7, bias=True)
  (maxpool): MaxPool2d(kernel_size=(53, 1), stride=(53, 1), padding=0, dilation=1, ceil_mode=False)
)
1: accuracy:0.2695035460992908 loss: 1.9541912078857422
2: accuracy:0.3900709219858156 loss: 1.772621750831604
3: accuracy:0.3333333333333333 loss: 1.606994390487671
4: accuracy:0.4326241134751773 loss: 1.4174365997314453
5: accuracy:0.5177304964539007 loss: 1.2522685527801514
6: accuracy:0.5673758865248227 loss: 1.1295371055603027
7: accuracy:0.5531914893617021 loss: 0.9108871817588806
8: accuracy:0.6028368794326241 loss: 0.7546970844268799
9: accuracy:0.574468085106383 loss: 0.5393237471580505
10: accuracy:0.6170212765957447 loss: 0.4772818088531494
11: accuracy:0.624113475177305 loss: 0.37971144914627075
12: accuracy:0.6312056737588653 loss: 0.25231868028640747
13: accuracy:0.6312056737588653 loss: 0.18982568383216858
14: accuracy:0.6382978723404256 loss: 0.11684030294418335
15: a

135: accuracy:0.6312056737588653 loss: 0.0005545839667320251
136: accuracy:0.6312056737588653 loss: 0.00046394020318984985
137: accuracy:0.6382978723404256 loss: 0.0004654601216316223
138: accuracy:0.6382978723404256 loss: 0.00037739425897598267
139: accuracy:0.6382978723404256 loss: 0.0004795156419277191
140: accuracy:0.6382978723404256 loss: 0.0004212707281112671
141: accuracy:0.6382978723404256 loss: 0.0005138963460922241
142: accuracy:0.6382978723404256 loss: 0.00042288750410079956
143: accuracy:0.6382978723404256 loss: 0.00043049082159996033
144: accuracy:0.6382978723404256 loss: 0.000390518456697464
145: accuracy:0.6382978723404256 loss: 0.0003867931663990021
146: accuracy:0.6382978723404256 loss: 0.00042531639337539673
147: accuracy:0.6382978723404256 loss: 0.00036405399441719055
148: accuracy:0.6382978723404256 loss: 0.00033909454941749573
149: accuracy:0.6382978723404256 loss: 0.00040668994188308716
150: accuracy:0.6382978723404256 loss: 0.00033704936504364014
151: accuracy:0.